## Bronze Load - Yellow Taxi

In [0]:
dbutils.library.restartPython()


In [0]:
import sys

sys.path.insert(0, "../lib/")

from utils import import_schema, export_schema, table_exists


In [0]:
catalog = "bronze"
schema = "taxi" 
table_name = "yellow_taxi"

df_schema = import_schema(table_name)
path = f"/Volumes/raw/{schema}/{table_name}"

for year in range(2013, 2018):

      df = (spark.read
            .format('parquet')
            .schema(df_schema)
            .load(f"{path}/{year}"))
      if not table_exists(spark, catalog, schema, table_name):

            (df.write
                  .format("delta")
                  .mode("overwrite")
                  .saveAsTable(f"{catalog}.{schema}.{table_name}"))
      else:
            (df.write
                  .format("delta")
                  .mode("append")
                  .saveAsTable(f"{catalog}.{schema}.{table_name}"))



In [0]:
from pyspark.sql.functions import col

# year = '2020/yellow_tripdata_2020-12.parquet'

for year in range(2021,2023):
        df = (spark.read
                .format('parquet')
                # .option("mergeSchema", "true")
                .load(f"{path}/{year}"))

        # Realiza o cast de colunas para evitar erro
        df = (df.withColumn("airport_fee", col("airport_fee").cast("int")))
        df = (df.withColumn("congestion_surcharge", col("congestion_surcharge").cast("double")))
        df = (df.withColumn("passenger_count", col("passenger_count").cast("long")))
        df = (df.withColumn("RatecodeID", col("RatecodeID").cast("long")))


        if not table_exists(spark, catalog, schema, table_name):
                (df.write
                        .format("delta")
                        .mode("overwrite")
                        .saveAsTable(f"{catalog}.{schema}.{table_name}"))
        else:
                (df.write
                        .format("delta")
                        .mode("append")
                        .saveAsTable(f"{catalog}.{schema}.{table_name}"))


In [0]:
%sql
SELECT COUNT(*) FROM bronze.taxi.yellow_taxi 
-- 891.101.751

In [0]:
# export_schema(spark, '2017-12')
# export_schema(spark, '2020-08')